In [1]:
# Load Library
from pandas import DataFrame
import pandas as pd
import numpy as np
from Bio import Entrez 
from Bio import SeqIO
import pycurl
import matplotlib.pyplot as plt
import os, sys
from tqdm.notebook import trange, tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

In [37]:
try:
    df = pd.read_csv("output/ncov2019_ncbi_spike.csv", index_col=0)
except:
    df = pd.read_csv("dataset/ncov2019_ncbi08052020.csv")
    df["s-gene_location"] = ""
    df["gene_length"] = ""
    df["protein_id"] = ""
    df["protein_length"] = ""

In [38]:
df

,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,s-gene_location,gene_length,protein_id,protein_length
0,NC_045512,2020-01-13T00:00:00Z,Severe acute respiratory syndrome-related coro...,29903,China,Homo sapiens,NaN,2019-12,[21562:25384](+),3822.0,YP_009724390.1,1273.0
1,MT444148,2020-05-08T00:00:00Z,Severe acute respiratory syndrome-related coro...,29890,USA: CA,Homo sapiens,oronasopharynx,2020-03-11,[21549:25371](+),3822.0,QJQ82624.1,1273.0
2,MT444515,2020-05-08T00:00:00Z,Severe acute respiratory syndrome-related coro...,29804,USA,Homo sapiens,NaN,2020-03-16,[21530:25352](+),3822.0,QJQ82636.1,1273.0
3,MT444516,2020-05-08T00:00:00Z,Severe acute respiratory syndrome-related coro...,29786,USA,Homo sapiens,NaN,2020-03-16,[21512:25334](+),3822.0,QJQ82648.1,1273.0
4,MT444517,2020-05-08T00:00:00Z,Severe acute respiratory syndrome-related coro...,29797,USA,Homo sapiens,NaN,2020-03-16,[21523:25345](+),3822.0,QJQ82660.1,1273.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2429,MN975268,2020-01-24T00:00:00Z,Severe acute respiratory syndrome-related coro...,107,China,Homo sapiens,"lung, oronasopharynx",2020-01,[<0:>107](+),107.0,QHN73824.1,35.0
2430,MN985325,2020-01-24T00:00:00Z,Severe acute respiratory syndrome-related coro...,29882,USA,Homo sapiens,oronasopharynx,2020-01-19,[21562:25384](+),3822.0,QHO60594.1,1273.0
2431,MN970003,2020-01-23T00:00:00Z,Severe acute respiratory syndrome-related coro...,290,Thailand,Homo sapiens,"lung, oronasopharynx",2020-01-08,NaN,NaN,NaN,NaN
2432,MN970004,2020-01-23T00:00:00Z,Severe acute respiratory syndrome-related coro...,290,Thailand,Homo sapiens,"lung, oronasopharynx",2020-01-13,NaN,NaN,NaN,NaN


In [29]:
#Email for NCBI
Entrez.email = "matin_nuhamunada@ugm.ac.id"

for i in tqdm(df.index):
    if df.loc[i, "s-gene_location"] == '':
        handle = Entrez.efetch(db="nucleotide", id=df.Accession[i], rettype="gb", retmode="text")
        seq_record = SeqIO.read(handle, "genbank")
        handle.close()
        #print(seq_record.id)
        for feature in seq_record.features:
            try:
                if feature.type == 'CDS':
                    if feature.qualifiers['gene'][0] == 'S':
                        start = feature.location.start.position
                        end = feature.location.end.position
                        protein_id = str(feature.qualifiers['protein_id'][0])
                
                        df.loc[i, "s-gene_location"] = str(feature.location)
                        df.loc[i, "gene_length"] = len(seq_record.seq[start:end])
                        df.loc[i, "protein_id"] = protein_id
                        df.loc[i, "protein_length"] = len(feature.qualifiers['translation'][0])
                
                    #write spike gene fasta
                        record = SeqRecord(Seq(""))
                        record.seq = seq_record.seq[start:end]
                        record.id = seq_record.id
                        with open("output/gene/s_gene_"+str(i)+"_"+record.id+".fasta", "w") as output_handle:
                            SeqIO.write(record, output_handle, "fasta")
                
                    #write spike protein fasta
                        protein = feature.qualifiers['translation'][0]
                        protein_record = SeqRecord(Seq(protein, IUPAC.protein), id= protein_id, description=feature.qualifiers['product'][0])
                        with open("output/protein/spike_"+str(i)+"_"+protein_id+".fasta", "w") as output_handle:
                            SeqIO.write(protein_record, output_handle, "fasta")
            except:
                print('error')

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [30]:
df.to_csv('output/ncov2019_ncbi_spike.csv')